In [2]:
import os, sys
import mujoco
import mujoco.viewer
import numpy as np
import time

In [4]:

# Load model to get nq and nv
model = mujoco.MjModel.from_xml_path("F:\\Sheffield's Folder\\CMU\\16-745 OPT-Control & RL\\Projects\\Humanoid_MPPI-RL\\Visualization\\humanoid.xml")
nq = model.nq
nv = model.nv

# Load your state file
data_directory = "F:\\Sheffield's Folder\\CMU\\16-745 OPT-Control & RL\\Projects\\Humanoid_MPPI-RL\\data\\2025-04-19_153833"
state_path = os.path.join(data_directory, "states.csv")
states = np.loadtxt(state_path, delimiter=",", skiprows=1)  # adjust if header
print(f"Loaded {states.shape[0]} frames, {states.shape[1]} features per state.")

# Validate and slice if necessary
assert states.shape[1] >= nq + nv, "State file doesn't contain enough dimensions for qpos + qvel"

qpos_qvel = states[:, :nq+nv]  # Take only the first nq+nv columns

# Save as trajectory.csv
np.savetxt("trajectory.csv", qpos_qvel, delimiter=",")
print("Saved trajectory.csv!")


Loaded 24031 frames, 55 features per state.
Saved trajectory.csv!


In [6]:
# Load the model
model = mujoco.MjModel.from_xml_path("F:\\Sheffield's Folder\\CMU\\16-745 OPT-Control & RL\\Projects\\Humanoid_MPPI-RL\\Visualization\\humanoid.xml")
data = mujoco.MjData(model)

# Dummy predicted trajectory (replace with your real output)
# Shape: (T, qpos_dim + qvel_dim)
trajectory = np.loadtxt("trajectory.csv", delimiter=",")  # ← or from your model

# Launch viewer
with mujoco.viewer.launch_passive(model, data) as viewer:
    for t in range(len(trajectory)):
        # Unpack qpos and qvel
        qpos_dim = model.nq
        qvel_dim = model.nv
        qpos = trajectory[t, :qpos_dim]
        qvel = trajectory[t, qpos_dim:qpos_dim+qvel_dim]

        # Set state
        data.qpos[:] = qpos
        data.qvel[:] = qvel

        # Forward and render
        mujoco.mj_forward(model, data)
        viewer.sync()

        time.sleep(1 / 60.0)  # 60 FPS


KeyboardInterrupt: 